This notebook is written by the DAWS2 UQ team based in Univ. of Liverpool for the workshop of UQ & M of the DAWS2 project.

Uncertainty Quantification & Management
--


DAWS2 UQ team: {Scott Ferson, Ioanna,  [(Leslie) Yu Chen](https://yuchenakaleslie.github.io/)}

All rights reserved.

===================

## Table of Contents
#### [1. Setup](#Setup)
#### [2. Problem statement](#problem_statement)
#### [3. Create propagating function](#create_propagating_function)
#### [4. Epistemic uncertainty](#propagate_epistemic_uncertainty)
#### <small>[4.1. Uncertainty Characterisation](#4.1-uncertainty-characterisation)</small>
#### <small>[4.2. Uncertainty propagation methods](#4.2-uncertainty-propatation-methods)</small>
##### <small>[4.2.1. Endpoints propagation](#4.2.1-Endpoints-propatation)</small>
##### <small>[4.2.2. Subinterval reconstitution propagation](#4.2.2.-Subinterval-reconstitution-propagation)</small>
##### <small>[4.2.3. Sampling propagation](#4.2.3.-Sampling-propagation)</small>
##### <small>[4.2.4. Optimisation methods](#4.2.3.-Optimisation-methods)</small>
#### [5. Aleatory uncertainty](#aleatory_uncertainty_propagation)
#### [5. Mixed types of uncertainty](#mixed_uncertainty_propagation)

***
<a id="setup"></a>
# 1. Setup

Import the libraries

In [9]:
import numpy as np
from pyuncertainnumber.propagation.uncertaintyPropagation import Propagation, epistemic_propagation, aleatory_propagation, mixed_propagation
from pyuncertainnumber import UncertainNumber as UN
import matplotlib.pyplot as plt

In [10]:
%load_ext autoreload
%autoreload 2
%load_ext rich

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The rich extension is already loaded. To reload it, use:
  %reload_ext rich


In [11]:
# # import user-defined function

# # USER_dir = 'C:\\Users\\Ioanna\\Documents\\GitHub\\PyUncertainNumber\\cantilever_beam'
# import sys
# sys.path.append(USER_dir)

***
<a id="problem_statement"></a>
# 2. Problem statement

The numerical example models a simple cantilever beam with length, $L$, distance to the neutral axis $y$, Young’s modulus, $E$, second moment of inertia, $I$, and external load, $F$. We will compute the bending stress $σ$, and the deflection $d$, assuming the above input parameters are **uncertain numbers**.

We will use this example throughout the document to illustrate the differences among various uncertainty quantification approaches. A complete description of the following methods can be found in the [DAWS1 report](https://sites.google.com/view/dawsreports/up).

![alt text](../assets/cantilever.png)

**Fig.1** Cantilever beam with input parameters.  

***
<a id="create_propagating_function"></a>
# 3. Create the propagating function


A propagating function is created which calculates the deflection, $d$, as a function of $L$, $E$, $I$, and $F$.
- The function's input is only the uncertain numbers.
- As sampling techniques are used it is likely that for a certain input combination the airfoil will fail to reach a solution. we use the try function to accomodate for this.


Alternatively, such propating function can be imported as a Python function/method object from a certain local directory.

<a id="3.1.-create-function"></a>

## 3.1. Create propagating function

In [12]:
def cantilever_beam_deflection(x):
    """Calculates deflection and stress for a cantilever beam.

    Args:
        x (np.array): Array of input parameters:
            x[0]: Length of the beam (m)
            x[1]: Second moment of area (mm^4)
            x[2]: Applied force (N)
            x[3]: Young's modulus (MPa)

    Returns:
        float: deflection (m)
               Returns np.nan if calculation error occurs.
    """

    beam_length = x[0]
    I = x[1]
    F = x[2]
    E = x[3]
    # try:  # try is used to account for cases where the input combinations leads to error in fun due to bugs
    #     deflection = F * beam_length**3 / (3 * E * 10**6 * I)  # deflection in m
        
    # except:
    #     deflection = np.nan
    
    deflection = F * beam_length**3 / (3 * E * 10**6 * I)  # deflection in m
    return np.array([deflection])

In [13]:
# know the function with default Python help mechanism
cantilever_beam_deflection?

Signature: cantilever_beam_deflection(x)
Docstring:
Calculates deflection and stress for a cantilever beam.

Args:
    x (np.array): Array of input parameters:
        x[0]: Length of the beam (m)
        x[1]: Second moment of area (mm^4)
        x[2]: Applied force (N)
        x[3]: Young's modulus (MPa)

Returns:
    float: deflection (m)
           Returns np.nan if calculation error occurs.
File:      /var/folders/dr/wmmn28yj3ldb2nmth0yqp2mr0000gn/T/ipykernel_97263/3680808391.py
Type:      function

In [63]:
# import numpy as np 
# x = np.array([1.00500000e+01 , 3.86159100e-04 ,-8.24387293e+00 , 2.12872340e+02])
# y = cantilever_beam_deflection(x)
# print(y)

<a id="3.2.-Upload-function"></a>

## 3.2. Upload propagating function

In [15]:
# Alternatively, such propating function can be imported as a Python function/method object from a certain local directory.
from pyuncertainnumber.propagation.performance_func import cb_deflection, cb_stress

<a id="3.3.-verify-function"></a>

## 3.3. Verify the function

To ensure that the function yields meaningful results. We consider that input has the nominal values seen below.

The function should yield deflection equal to 0.162m.
 

In [16]:
# test the function

y  = 0.155 # m
L  = 10.05 # m
I = 0.000386 # m**4
F = 37 # kN
E = 200 # GPa

x = np.array([L, I, F, E])
deflection = cantilever_beam_deflection(x)

print(deflection) # 0.162m

[0.16216658]


***
<a id="propagate_epistemic_uncertainty"></a>
# 4. Epistemic uncertainty


Construct `UncertainNumbers` objects for the inputs assuming they are intervals with the lower and upper values are seen below

- $y = [0.145, 0.155] \ m$

- $L = [9.95, 10.05] \ m$

- $I = [0.0003861591, 0.0005213425] \ m^{4}$

- $F = [11, 37] \ kN$

- $E = [200, 220] \ GPa$


Choose from a suite of black box propagating techniques to propagate the intervals through the model. 

In [17]:
#y = UN(name='beam width', symbol='y', units='m', essence='interval', bounds=[0.145, 0.155]) # Required only for stress estimation
L = UN(name='beam length', symbol='L', units='m', essence='interval', bounds= [9.95, 10.05])
I = UN(name='moment of inertia', symbol='I', units='m', essence='interval', bounds= [0.0003861591, 0.0005213425])
F = UN(name='vertical force', symbol='F', units='kN', essence='interval', bounds= [11, 37])
E = UN(name='elastic modulus', symbol='E', units='GPa', essence='interval', bounds=[200, 220])

In [18]:
defl = cantilever_beam_deflection([L, I, F, E])

In [19]:
defl


array([UncertainNumber(essence='interval', bounds=Interval [0.0314916, 0.1621], _construct=Interval [0.0314916, 0.1621], naked_value=0.09679568741716935)],
      dtype=object)

<a id="4.2.1.-Endpoints-propatation"></a>

### 4.2.1. Endpoint propagation

The endpoint propagation method (Dong and Shah, 1987) is a straightforward way to project intervals through the code, by projecting all input combinations produced by the Cartesian product of the interval bounds. This results in a total of $n = 2^{d}$. 

For the working example, there are d = 5 intervals which results in $n = 2^{5} = 32$ input combinations.

#### Assumptions




- [x] Ioanna's original top-level UP function with  the 'endspoints' method

- [x] Leslie's standalone endspoints implementation

In [20]:
METHOD = "endpoints"

a = Propagation(vars=[L, I, F, E], 
          fun=cantilever_beam_deflection, 
          method = METHOD,
          save_raw_data = "no" 
        #   save_raw_data = "yes", 
        #   base_path = USER_dir
         )                 

Total number of input combinations for the endpoint method: 16


Evaluating samples: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 152520.15it/s]


In [23]:
a.un

UncertainNumber(essence='interval', bounds=array([0.03149161, 0.16209977]), _construct=Interval [0.0314916, 0.1621], naked_value=np.float64(0.09679568741716935))

<a id="4.2.2.-Subinterval-reconstitution-propagation"></a>

### 4.2.2. Subinterval reconstitution propagation

The input intervals are partitioned into smaller intervals, which are then propagated through the model using endpoint propagation and the output interval can be reassembled (Ferson and Hajagos, 2004).

In [24]:
METHOD = "subinterval"

a = Propagation(vars=[L, I, F, E], 
          fun=cantilever_beam_deflection, 
          method = METHOD,
          save_raw_data = "no" 
        #   save_raw_data = "yes", 
        #   base_path = USER_dir
         )        

Total number of input combinations for the subinterval method: 256


Evaluating samples: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [00:00<00:00, 535799.31it/s]


In [25]:
a.un

UncertainNumber(essence='interval', bounds=array([0.03149161, 0.16209977]), _construct=Interval [0.0314916, 0.1621], naked_value=np.float64(0.09679568741716935))

<a id="4.2.3.-Sampling-propagation"></a>

### 4.2.3. Sampling propagation

- Brute Monte Carlo
- Latin Hypercube
- Brute Monte Carlo + endpoints
- Latin Hypercuve + endpoints

In [26]:
METHOD = "monte_carlo"

a = Propagation(vars=[L, I, F, E], 
          fun=cantilever_beam_deflection,
          n_sam = 1000, 
          method = METHOD,
          save_raw_data = "no" 
        #   save_raw_data = "yes", 
        #   base_path = USER_dir
         )        

Total number of input combinations for the monte_carlo method: 1000


Evaluating samples: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 753016.88it/s]


In [29]:
a.un

UncertainNumber(essence='interval', bounds=array([0.03286865, 0.1567389 ]), _construct=Interval [0.0328686, 0.156739], naked_value=np.float64(0.09480377530913246))

In [31]:
METHOD = "monte_carlo"

b = epistemic_propagation(vars=[L, I, F, E], 
          fun=cantilever_beam_deflection,
          n_sam = 1000,  
          method = METHOD, 
          save_raw_data = "no"
        #   save_raw_data = "yes", 
        #   base_path = USER_dir
         )

Total number of input combinations for the monte_carlo method: 1000


Evaluating samples: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 724279.74it/s]


In [32]:
b.un

UncertainNumber(essence='interval', bounds=array([0.03535911, 0.14967499]), _construct=Interval [0.0353591, 0.149675], naked_value=np.float64(0.09251705139529531))

In [33]:
METHOD = "monte_carlo_endpoints"

c = Propagation(vars=[L, I, F, E], 
          fun=cantilever_beam_deflection,
          n_sam = 1000,  
          method = METHOD, 
          save_raw_data = "no"
        #   save_raw_data = "yes", 
        #   base_path = USER_dir
         )

Total number of input combinations for the monte_carlo method: 1000


Evaluating samples: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1016/1016 [00:00<00:00, 642165.89it/s]


In [34]:
c.un

UncertainNumber(essence='interval', bounds=array([0.03149161, 0.16209977]), _construct=Interval [0.0314916, 0.1621], naked_value=np.float64(0.09679568741716935))

In [35]:
METHOD = "monte_carlo_endpoints"

d = epistemic_propagation(vars=[L, I, F, E], 
          fun=cantilever_beam_deflection,
          n_sam = 1000,  
          method = METHOD, 
          save_raw_data = "no"
        #   save_raw_data = "yes", 
        #   base_path = USER_dir
         )

Total number of input combinations for the monte_carlo method: 1000


Evaluating samples: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1016/1016 [00:00<00:00, 644691.81it/s]


In [36]:
d.un

UncertainNumber(essence='interval', bounds=array([0.03149161, 0.16209977]), _construct=Interval [0.0314916, 0.1621], naked_value=np.float64(0.09679568741716935))

### 4.2.4. Sampling propagation

- Latin Hypercube
- Latin Hypercuve + endpoints

In [37]:
METHOD = "latin_hypercube"

a = Propagation(vars=[L, I, F, E], 
          fun=cantilever_beam_deflection,
          n_sam = 1000, 
          method = METHOD,
          save_raw_data = "no" 
        #   save_raw_data = "yes", 
        #   base_path = USER_dir
         )        

Total number of input combinations for the latin_hypercube method: 1000


Evaluating samples: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 954118.29it/s]


In [38]:
a.un

UncertainNumber(essence='interval', bounds=array([0.03315123, 0.15256283]), _construct=Interval [0.0331512, 0.152563], naked_value=np.float64(0.09285703033588368))

In [41]:
# METHOD = "latin_hypercube"

# b = epistemic_propagation(vars=['L', 'I', 'F', 'E'], 
#           fun=cantilever_beam_deflection,
#           n_sam = 1000, 
#           method = METHOD,
#           save_raw_data = "no" 
#         #   save_raw_data = "yes", 
#         #   base_path = USER_dir
#          )        

In [40]:
b.un

UncertainNumber(essence='interval', bounds=array([0.03535911, 0.14967499]), _construct=Interval [0.0353591, 0.149675], naked_value=np.float64(0.09251705139529531), _field_str="essence='interval', bounds=array([0.03535911, 0.14967499]), _construct=Interval [0.0353591, 0.149675], naked_value=np.float64(0.09251705139529531)")

In [42]:
METHOD = "latin_hypercube_endpoints"

c = Propagation(vars=[L, I, F, E], 
          fun=cantilever_beam_deflection,
          n_sam = 1000, 
          method = METHOD,
          save_raw_data = "no" 
        #   save_raw_data = "yes", 
        #   base_path = USER_dir
         )       

Total number of input combinations for the latin_hypercube method: 1000


Evaluating samples: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1016/1016 [00:00<00:00, 622558.49it/s]


In [43]:
c.un

UncertainNumber(essence='interval', bounds=array([0.03149161, 0.16209977]), _construct=Interval [0.0314916, 0.1621], naked_value=np.float64(0.09679568741716935))

In [44]:
METHOD = "latin_hypercube_endpoints"

d = epistemic_propagation(vars=[L, I, F, E], 
          fun=cantilever_beam_deflection,
          n_sam = 1000, 
          method = METHOD,
          save_raw_data = "no" 
        #   save_raw_data = "yes", 
        #   base_path = USER_dir
         )      

Total number of input combinations for the latin_hypercube method: 1000


Evaluating samples: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1016/1016 [00:00<00:00, 567658.57it/s]


In [47]:
d.un

UncertainNumber(essence='interval', bounds=array([0.03149161, 0.16209977]), _construct=Interval [0.0314916, 0.1621], naked_value=np.float64(0.09679568741716935))

<a id="4.2.5.-Optimisation-methors"></a>

### 4.2.5. Optimisation methods

- Local optimisation
- Genetic algorithm


In [53]:
# METHOD = "local_optimisation"

# a = Propagation(vars=[L, I, F, E], 
#        fun = cantilever_beam_deflection, 
#        x0 = None, 
#        method = METHOD, 
#        method_loc = 'Nelder-Mead')

In [54]:
a.un
a.print()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 a.un                                                                                         │
│ ❱ 2 a.print()                                                                                    │
│   3                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'Propagation_results' object has no attribute 'print'

In [ ]:
# METHOD = "local_optimisation"

# b = epistemic_propagation(vars=[L, I, F, E], 
#        fun = cantilever_beam_deflection, 
#        #x0 = None, 
#        method = METHOD, 
#        method_loc = 'Nelder-Mead')
# b.print()
# b.un

In [55]:
METHOD = "genetic_optimisation"

a = Propagation(vars=[L, I, F, E], 
       fun = cantilever_beam_deflection, 
       method = METHOD)

Convergence reached!
Convergence reached!


In [59]:
a.un

UncertainNumber(essence='interval', bounds=array([0.0314993 , 0.16208952]), _construct=Interval [0.0314993, 0.16209], naked_value=np.float64(0.0967944098712922))

***
<a id="aleatory_uncertainty_propagation"></a>
# 5. Aleatory uncertainty

when inputs have various types of uncertainty.

In [60]:
# y = UN(name='beam width', symbol='y', units='m', essence='interval', bounds=[0.145, 0.155]) # Required only for stress estimation
L = UN(name='beam length', symbol='L', units='m', essence='distribution', distribution_parameters=["gaussian", [10.05, 0.033]])
I = UN(name='moment of inertia', symbol='I', units='m', essence='distribution', distribution_parameters=["gaussian", [0.000454, 4.5061e-5]])
F = UN(name='vertical force', symbol='F', units='kN', essence='distribution', distribution_parameters=["gaussian", [24, 8.67]])
E = UN(name='elastic modulus', symbol='E', units='GPa', essence='distribution', distribution_parameters=["gaussian", [210, 6.67]])

In [61]:
METHOD = "monte_carlo"

a = Propagation(vars=[L, I, F, E], 
          fun = cantilever_beam_deflection, 
          n_sam = 100, 
          method = METHOD, 
          save_raw_data = "no"
         )


Total number of input combinations for the monte_carlo method: 100


Evaluating samples: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 855980.41it/s]


In [62]:
a.un

[dist ~ sample-approximated distribution object]

***
<a id="mixed_uncertainty_propagation"></a>
# 6. Mixed types of uncertainty

when inputs have various types of uncertainty.

# Naive pbox arithmetic

In [56]:
# y = UN(name='beam width', symbol='y', units='m', essence='interval', bounds=[0.145, 0.155]) # Required only for stress estimation
L = UN(name='beam length', symbol='L', units='m', essence='distribution', distribution_parameters=["gaussian", [10.05, 0.033]])
I = UN(name='moment of inertia', symbol='I', units='m', essence='distribution', distribution_parameters=["gaussian", [0.000454, 4.5061e-5]])
F = UN(name='vertical force', symbol='F', units='kN', essence='distribution', distribution_parameters=["gaussian", [24, 8.67]])
E = UN(name='elastic modulus', symbol='E', units='GPa', essence='distribution', distribution_parameters=["gaussian", [210, 6.67]])

In [57]:
defl = cantilever_beam_deflection([L, I, F, E])

In [58]:
defl


array([UncertainNumber(essence='pbox', _construct=Pbox: ~ (range=['-0.05', '0.37'], mean=[0.0002, 0.0045], var=[0.0002, 0.0045]))],
      dtype=object)